In [ ]:
# config variables
from azure.identity import DefaultAzureCredential

credential=DefaultAzureCredential()
resource_group_name = "dantaylo_sdktest_rg3"
hub_name = "dantaylo_sdktest_hub3"
project_name = "dantaylo-sdktest-project3"
location = "eastus"
subscription_id = "597966d1-829f-417e-9950-8189061ec09c"
aiservices_resource_name = "dantaylo_sdktest_aiservices3"
aisearch_resource_name = "dantaylo-sdktest-aisearch3"
aisearch_index_name = "dantaylo_aisearch_index"
azure_openai_chat_model = "gpt-35-turbo"
azure_openai_embedding_model = "text-embedding-ada-002"

In [ ]:
# Build index
from config import resource_group_name, project_name, subscription_id, credential,azure_openai_embedding_model_name, aisearch_index_name
from promptflow.rag.resources import LocalSource, AzureAISearchConfig, EmbeddingsModelConfig, ConnectionConfig
from promptflow.rag import build_index

  # Use the same index name when registering the index in AI Studio
index_path = build_index(
    name=aisearch_index_name,  # name of your index
    vector_store="azure_ai_search",  # the type of vector store
    embeddings_model_config=EmbeddingsModelConfig(
        model_name=azure_openai_embedding_model_name, # the name of the model
        deployment_name=azure_openai_embedding_model_name,
        connection_config= ConnectionConfig(
            subscription=subscription_id,
            resource_group=resource_group_name,
            workspace=project_name,
            connection_name="azure_aiservices_connection"
        )
    ),
    input_source=LocalSource(input_data="./data/*.md"),  # the location of your file/folders
    index_config=AzureAISearchConfig(
            ai_search_index_name=index_name, # the name of the index store inside the azure ai search service
            ai_search_connection_config=ConnectionConfig(
            subscription=subscription_id,
            resource_group=resource_group_name,
            workspace=project_name,
            connection_name="azure_aisearch_connection"
        ) # this is so that you do not need the environment variables
    ),
    tokens_per_chunk = 800, # Optional field - Maximum number of tokens per chunk
    token_overlap_across_chunks = 0, # Optional field - Number of tokens to overlap between chunks
)

from promptflow.rag import get_langchain_retriever_from_index

# Call the index
retriever=get_langchain_retriever_from_index(ml_index.path)
retriever.get_relevant_documents("which tent is the most waterproof?")

# register the index so that it shows up in the project
from azure.ai.ml import MLClient
client.indexes.create_or_update(Index(name=index_name, path=index_path))
client = MLClient(workspace_name=project_name, 
    resource_group_name=resource_group_name, 
    subscription_id=subscription_id, 
    credential=credential)


In [ ]:
from promptflow.core import tool, Prompty

# load and execute prompty
path_to_prompty =  "basic_chat.prompty"
flow = Prompty.load(path_to_prompty)

# execute the flow as function
result = flow(question=question)
print(result)



In [ ]:
import os
from promptflow.core import AzureOpenAIModelConfiguration

# Initialize Azure OpenAI Connection
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
)

# Run built-in evaluator on a single input row
from promptflow.evals.evaluators import RelevanceEvaluator

relevance_eval = RelevanceEvaluator(model_config)
relevance_score = relevance_eval(
    answer="The Alpine Explorer Tent is the most waterproof.",
    context="From the our product list,"
    " the alpine explorer tent is the most waterproof."
    " The Adventure Dining Table has higher weight.",
    question="Which tent is the most waterproof?",
)
# relevance_score: 5.0

# custom evaluator
from promptflow.client import load_flow

# load apology evaluatorfrom prompty
apology_eval = load_flow(source="apology.prompty", model={"configuration": model_config})

result = apology_eval(
    question="What is the capital of France?", answer="Paris"
)

# result: {'apology': 0}


from promptflow.evals.evaluate import evaluate

result = evaluate(
    data="data.jsonl",
    evaluators={
        "relevance": relevance_eval,
        "violence": violence_eval,
        "answer_length": answer_length,
        "apology": apology_eval
    },
    # column mapping
    evaluator_config={
        "default": {
            "ground_truth": "${data.truth}"
        }
    }
)